In [ ]:
# What these guardrails protect against (in simple words)
# Schema → “Output must look like this, not random text”
# Validation → “Check if output actually meets expectations”
# Retries → “If output is bad, try again automatically”
# Logging → “Track what happened and debug easily”

"""
CrewAI – Hierarchical Multi-Agent System
WITH Guardrails:
- Output schema
- Validation
- Retries
- Logging
"""

# ============================================================
# 0) STANDARD IMPORTS
# ============================================================
from dotenv import load_dotenv
load_dotenv()

import os
import logging
from typing import List
from pydantic import BaseModel, Field, ValidationError

from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# ============================================================
# 1) LOGGING SETUP (Guardrail #1)
# ============================================================
# This logs every important step and error
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
 )

logger = logging.getLogger(__name__)

# ============================================================
# 2) OUTPUT SCHEMAS (Guardrail #2)
# ============================================================
# These schemas define EXACTLY how outputs should look

class ResearchOutput(BaseModel):
    facts: List[str] = Field(
        min_items=3,
        max_items=5,
        description="3–5 recent factual bullet points"
    )

class BlogOutput(BaseModel):
    blog_summary: str = Field(
        min_length=80,
        max_length=130,
        description="~100 word blog summary"
    )

# ============================================================
# 3) LLM CONFIG
# ============================================================
llm = LLM(
    model="gpt-4o",
    temperature=0.7
)

# ============================================================
# 4) TOOLS
# ============================================================
search_tool = SerperDevTool()

# ============================================================
# 5) AGENTS
# ============================================================
research_agent = Agent(
    role="Research Specialist",
    goal="Find accurate, recent, verifiable facts about {topic}",
    backstory="Expert internet researcher focused on credibility and recency.",
    tools=[search_tool],
    llm=llm,
    verbose=True
)

writer_agent = Agent(
    role="Creative Writer",
    goal="Write a concise blog using ONLY verified research",
    backstory="Professional writer who avoids hallucinations.",
    llm=llm,
    verbose=True
)

manager_agent = Agent(
    role="Editor-in-Chief (Manager)",
    goal="Ensure factual accuracy, structure, and schema compliance",
    backstory="Senior editor who validates quality before publishing.",
    llm=llm,
    allow_delegation=True,
    verbose=True
)

# ============================================================
# 6) TASKS (WITH SCHEMA EXPECTATION)
# ============================================================
task1 = Task(
    description=(
        "Find 3-5 interesting and recent facts about {topic} as of 2025. "
        "Return ONLY bullet points."
    ),
    expected_output="JSON list of facts as an object with a 'facts' field",
    agent=research_agent,
    #output_json=True
)

task2 = Task(
    description=(
        "Write a ~100 word blog summary using ONLY the researched facts."
    ),
    expected_output="JSON with a blog_summary field",
    agent=writer_agent,
    context=[task1],
    #output_json=True
)

# ============================================================
# 7) CREW (HIERARCHICAL + MEMORY)
# ============================================================
crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[task1, task2],
    process=Process.hierarchical,
    manager_agent=manager_agent,
    verbose=True,
    memory=True,
    embedder={
        "provider": "openai",
        "config": {
            "api_key": os.getenv("OPENAI_API_KEY"),
            "model": "text-embedding-3-small"
        }
    }
)

# ============================================================
# 8) VALIDATION FUNCTION (Guardrail #3)
# ============================================================
def validate_outputs(result: dict):
    """
    Validates agent outputs against schemas.
    Raises ValidationError if anything is wrong.
    """
    logger.info("Validating research output...")
    ResearchOutput(**result["task_outputs"][0])

    logger.info("Validating blog output...")
    BlogOutput(**result["task_outputs"][1])

# ============================================================
# 9) RETRY MECHANISM (Guardrail #4)
# ============================================================
def run_with_retries(inputs: dict, max_retries: int = 3):
    for attempt in range(1, max_retries + 1):
        try:
            logger.info(f"Crew run attempt {attempt}")
            result = crew.kickoff(inputs=inputs)

            validate_outputs(result)
            logger.info("Validation successful")
            return result

        except ValidationError as ve:
            logger.error(f"Validation failed on attempt {attempt}: {ve}")

        except Exception as e:
            logger.error(f"Unexpected error on attempt {attempt}: {e}")

    raise RuntimeError("Crew failed after maximum retries")

# ============================================================
# 10) EXECUTION
# ============================================================
if __name__ == "__main__":
    final_result = run_with_retries(
        inputs={"topic": "The future of electrical vehicles"}
    )

    print("\n================ FINAL GUARDED OUTPUT ================\n")
    print(final_result)
